In [ ]:
SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME


In [ ]:
# Import python packages
import logging
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

current_context_df = cells.sql_get_context.to_pandas()
database_name = current_context_df.iloc[0,0]
schema_name = current_context_df.iloc[0,1]

session = get_active_session()
session.use_schema(f"{database_name}.{schema_name}")

logger.info("02_raw_to_harmonized start")

In [ ]:
def table_exists(session, database_name='', schema_name='', table_name=''):
    exists = session.sql("SELECT EXISTS (SELECT * FROM {}.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}') AS TABLE_EXISTS".format(database_name, schema_name, table_name)).collect()[0]['TABLE_EXISTS']
    return exists

In [ ]:
import snowflake.snowpark.functions as F

def create_fred_view(session):
    raw_fred_data = session.table("RAW_FRED.FRED_10Y_2Y")

    # Standardizing timestamps and financial indicators
    harmonized_data = raw_fred_data.select(
        F.to_date(F.col("observation_date")).alias("OBSERVATION_DATE"),
        F.col("T10Y2Y").cast("float").alias("YIELD_SPREAD")
    )

    harmonized_data = harmonized_data.drop_duplicates(["OBSERVATION_DATE"])

    # Step 4: Ensure data consistency (e.g., filter out invalid rows)
    harmonized_data = harmonized_data.filter(
        (F.col("YIELD_SPREAD").isNotNull())  # Remove rows with null values

    harmonized_data.create_or_replace_view('FRED_FLATTENED_V')

def create_fred_view_stream(session):
    _ = session.sql('CREATE OR REPLACE STREAM FRED_FLATTENED_V_STREAM \
                        ON VIEW FRED_FLATTENED_V \
                        SHOW_INITIAL_ROWS = TRUE').collect()
    


In [ ]:
create_fred_view(session)
create_fred_view_stream(session)
# logger.info(f"Successfully updated {table_name}")